In [1]:
from pyrdf2vec import RDF2VecTransformer
from pyrdf2vec.embedders import Word2Vec
from pyrdf2vec.graphs import KG, Vertex
from pyrdf2vec.walkers import RandomWalker
from rdflib import Graph
import pickle

In [2]:
g = Graph()
g.parse("../14_graph.nt", format="turtle")

<Graph identifier=N56e3c75741f2486a9e3663eec3ad4537 (<class 'rdflib.graph.Graph'>)>

In [3]:
triplets, uri_to_label, label_to_uri, label_list = pickle.load(open("../factual/formatted_data.pkl", "rb"))

In [4]:
CUSTOM_KG = KG()
entities = set()
for s, p, o in g:
    try:
        subject = Vertex(uri_to_label[str(s)])
    except:
        subject = Vertex(str(s))
    try:
        object = Vertex(uri_to_label[str(o)])
    except:
        object = Vertex(str(o))
    try:
        predicate = Vertex(uri_to_label[str(p)], predicate=True, vprev=subject, vnext=object)
    except:
        predicate = Vertex(str(p), predicate=True, vprev=subject, vnext=object)

    try:
        entities.add(uri_to_label[str(s)])
    except:
        entities.add(str(s))
    try:
        entities.add(uri_to_label[str(o)])
    except:
        entities.add(str(o))

    CUSTOM_KG.add_walk(subject, predicate, object)

In [5]:
entities

{'nm2244852',
 'tt9787732',
 '1981 film by Tommy Chong',
 'http://www.wikidata.org/entity/Q2113260',
 'Noon Orsatti',
 'Peyton Reed',
 'Brandon Scott',
 'tt7368116',
 'Raquel Welch',
 'nm0808129',
 'nm0600102',
 'Ann Sheridan',
 'Myriam Boyer',
 '2005 American horror-comedy film by Wes Craven',
 'Walter Slezak',
 'nm0570408',
 'Anna Gavalda',
 'Whitewater Films',
 'Tony Orlando',
 '1970 film by François Truffaut',
 'nm0949927',
 'fictional Universe',
 'nm0222150',
 'American actor (1908-1983)',
 'Romanian-American opera director',
 'nm2013400',
 'Jean-Paul Bonnaire',
 'nm0911125',
 'Metroville',
 'Eduardo Palomo',
 '2017 superhero film by Emilio Insolera - deaf film',
 'specific edition of a film festival, e.g. in a given year of an annual festival',
 'Steven Martini',
 'Christopher Cain',
 '1996-03-29',
 'tt0103253',
 'nm0908534',
 'nm0584974',
 'Duncan Chow',
 'Ryūichi Azuma',
 '2005-05-17',
 'Hope Marie Carlton',
 'nm0765533',
 'Seoul',
 'Chico Anysio',
 'Jon Kane',
 'Jean Acker',
 

In [6]:
transformer = RDF2VecTransformer(
    Word2Vec(epochs=10),
    walkers=[RandomWalker(4, 10, with_reverse=False, n_jobs=2)]
)

In [7]:
# Use GPU if available
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

embeddings = transformer.fit_transform(CUSTOM_KG, entities=entities)


embeddings = transformer.fit_transform(CUSTOM_KG, entities=entities)

KeyboardInterrupt: 

In [ ]:
# Save the embeddings
pickle.dump(embeddings, open("embeddings.pkl", "wb"))

In [ ]:
# query the embeddings for "Inception", "director" and find the closest entity from the sum of the embeddings
# of the two

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

inception = embeddings["Inception"]
director = embeddings["director"]
inception_director = inception + director

cosine_similarities = cosine_similarity([inception_director], embeddings)
closest = np.argmax(cosine_similarities)

# convert closest index to entity
entity = list(entities)[closest]
entity

